<a href="https://colab.research.google.com/github/aggarwal-ujjwal/AI-ML/blob/main/Deep_Q_Learning_for_Lunar_Landing_Handwritten_Code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://gymnasium.farama.org/environments/box2d/lunar_lander/

In [ ]:
!pip install gymnasium
!pip install "gymnasium[atari, accept-rom-license]"
!apt-get install -y swig
!pip install gymnasium[box2d]

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  swig4.0
Suggested packages:
  swig-doc swig-examples swig4.0-examples swig4.0-doc
The following NEW packages will be installed:
  swig swig4.0
0 upgraded, 2 newly installed, 0 to remove and 35 not upgraded.
Need to get 1,116 kB of archives.
After this operation, 5,542 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 swig4.0 amd64 4.0.2-1ubuntu1 [1,110 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/universe amd64 swig all 4.0.2-1ubuntu1 [5,632 B]
Fetched 1,116 kB in 3s (414 kB/s)
Selecting previously unselected package swig4.0.
(Reading database ... 126435 files and directories currently installed.)
Preparing to unpack .../swig4.0_4.0.2-1ubuntu1_amd64.deb ...
Unpacking swig4.0 (4.0.2-1ubuntu1) ...
Selecting previously unselected package swig.
Preparing to unpack .../swig_4.0.2-1ubunt

In [ ]:
import os
import random
import torch
import numpy as np
import torch.nn as nn
import torch.optim as optim
import torch.autograd as autograd
import torch.nn.functional as F
from collections import deque, namedtuple



Creating the NN

In [7]:
class NeuralNetwork(nn.Module): #inheritance
  def __init__(self, input_size, action_size, seed = 42): #you can remove seed from here
    super(NeuralNetwork, self).__init__()
    self.seed = torch.manual_seed(seed)
    self.fc1 = nn.Linear(input_size, 64) #64 has to be calculated with expermentation
    self.fc2 = nn.Linear(64,64)
    self.fc3 = nn.Linear(64, action_size) #ends with 4 possible actions
  #noob way
  # def forward(self, state):
  #   x = self.fc1(state)
  #   x = F.relu(x)
  #   x = self.fc2(x)
  #   x = F.relu(x)
  #   return self.fc3(x)

  #better way
  def forward(self, input): #input vector with 8 values like- ([0.5, -0.2, 1.0, 0.8])
    """ # fc1(state): [8] -> [64] (linear transformation + bias)
        # F.relu(): applies ReLU activation (max(0, x))
        # x shape: [64] with all negative values clipped to 0"""
    x = F.relu(self.fc1(input))
    x = F.relu(self.fc2(x))
    actions = self.fc3(x)
    return actions


In [13]:
#Example for understanding - not to be used anywhere
x = torch.tensor([1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0])
fc = nn.Linear(10, 64)
output = fc(x)
print(output)
print(x.shape, output.shape)
#Transforms input features into a higher-dimensional representation (10 → 64), allowing the network to learn more complex patterns.

tensor([-2.9912e-01, -3.4230e+00,  3.5476e+00,  2.2314e+00, -5.6016e+00,
         2.5683e+00,  1.2996e+00, -2.8921e+00, -2.7405e+00, -2.0260e+00,
         6.8871e+00, -3.4501e+00, -1.0745e+01, -2.0268e+00,  1.1376e+00,
        -4.9283e-01,  4.7010e-01,  2.5421e+00, -1.6787e+00, -4.8147e+00,
        -9.8752e-01, -9.9501e+00, -9.3120e+00,  1.4174e+00,  4.7613e+00,
         1.2533e+00,  3.4996e+00, -2.8903e+00,  1.8413e+00,  3.6027e-01,
        -1.9338e+00, -3.8423e+00,  1.9923e+00, -2.8707e+00,  1.1167e+00,
         3.4304e+00,  2.0469e+00,  1.6896e+00, -1.6622e+00, -4.7463e-01,
         3.0907e+00,  1.1544e+00,  2.5109e+00, -1.3669e+00, -2.1406e-01,
        -3.3151e+00, -2.3495e+00, -2.3412e+00,  3.0829e+00,  5.7850e+00,
         4.1797e+00,  5.5170e+00, -5.9128e+00,  4.0458e+00, -4.6105e+00,
        -2.9455e+00,  3.3453e+00,  1.0292e-02, -6.8615e+00,  2.3503e+00,
        -4.5343e+00,  1.1474e+00,  2.4058e+00, -1.1361e+00],
       grad_fn=<ViewBackward0>)
torch.Size([10]) torch.Size([64

Training the NN


In [19]:
import gymnasium as gym
env = gym.make('LunarLander-v3') # will get it from the documentation
print(env.action_space) #action - moving in 4 directions
print(env.observation_space) #input - 8 values

input_size = env.observation_space.shape[0]
action_size = env.action_space.n

print(input_size, action_size)



Discrete(4)
Box([ -2.5        -2.5       -10.        -10.         -6.2831855 -10.
  -0.         -0.       ], [ 2.5        2.5       10.        10.         6.2831855 10.
  1.         1.       ], (8,), float32)
8 4
